In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Craniosynostosis"
cohort = "GSE27976"

# Input paths
in_trait_dir = "../../input/GEO/Craniosynostosis"
in_cohort_dir = "../../input/GEO/Craniosynostosis/GSE27976"

# Output paths
out_data_file = "../../output/preprocess/Craniosynostosis/GSE27976.csv"
out_gene_data_file = "../../output/preprocess/Craniosynostosis/gene_data/GSE27976.csv"
out_clinical_data_file = "../../output/preprocess/Craniosynostosis/clinical_data/GSE27976.csv"
json_path = "../../output/preprocess/Craniosynostosis/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import os
import pandas as pd
import numpy as np
import json
import re
from typing import Optional, Callable, Dict, Any, List, Union

# Sample characteristics from previous output
sample_characteristics = {
    0: ['age months: 12.87', 'age months: 10.4', 'age months: 12.3', 'age months: 11.4', 'age months: 10.1', 'age months: 11', 'age months: 4.27', 'age months: 7.97', 'age months: 4.33', 'age months: 9.33', 'age months: 7.93', 'age months: 10.27', 'age months: 10.87', 'age months: 3.87', 'age months: 3.2', 'age months: 13.27', 'age months: 5.6', 'age months: 14.9', 'age months: 3.03', 'age months: 12.4', 'age months: 8.9', 'age months: 14.17', 'age months: 6.33', 'age months: 14.87', 'age months: 8.4', 'age months: 9.07', 'age months: 13.33', 'age months: 10', 'age months: 13.23', 'age months: 10.33'],
    1: ['gender: F', 'gender: M'],
    2: ['type: Metopic Synostosis', 'type: Coronal Synostosis R', 'type: Sagittal Synostosis', 'type: Coronal Synostosis L', 'type: Control'],
    3: ['cell lines: osteoblast'],
    4: ['tissue: skull']
}

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data for craniosynostosis patients
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# Examining the sample characteristics dictionary:

# 2.1 Data Availability
# Trait data is in row 2 - as "type" which indicates craniosynostosis type
trait_row = 2

# Age data is in row 0 - as "age months"
age_row = 0

# Gender data is in row 1 - as "gender"
gender_row = 1

# 2.2 Data Type Conversion Functions

def convert_trait(value: str) -> int:
    """
    Convert craniosynostosis type to binary (0=control, 1=case)
    """
    if pd.isna(value) or not isinstance(value, str):
        return None
    
    # Extract the value after the colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if "Control" in value:
        return 0
    elif "Synostosis" in value:
        return 1
    else:
        return None

def convert_age(value: str) -> float:
    """
    Convert age in months to a continuous value
    """
    if pd.isna(value) or not isinstance(value, str):
        return None
    
    # Extract the value after the colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Extract the numeric part
    match = re.search(r'(\d+\.?\d*)', value)
    if match:
        return float(match.group(1))
    else:
        return None

def convert_gender(value: str) -> int:
    """
    Convert gender to binary (0=female, 1=male)
    """
    if pd.isna(value) or not isinstance(value, str):
        return None
    
    # Extract the value after the colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if value.upper() == 'F':
        return 0
    elif value.upper() == 'M':
        return 1
    else:
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Initial validation and recording of metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# If trait data is available, extract and process clinical features
if trait_row is not None:
    # Create a suitable dataframe structure for the geo_select_clinical_features function
    # We need to ensure the structure works with get_feature_data called inside geo_select_clinical_features
    
    # The geo_select_clinical_features expects a dataframe where:
    # - Each row corresponds to a feature (age, gender, trait)
    # - The values should be unique values for that feature
    clinical_data = pd.DataFrame(sample_characteristics)
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the selected clinical features
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:", preview)
    
    # Create the directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical data to a CSV file
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
I understand that we need to properly analyze the dataset to find gene expression data availability and extract clinical features. Here's the corrected code:

```python
import pandas as pd
import numpy as np
import os
import json
from typing import Callable, Optional, Dict, Any, Union, List

# Load the necessary data
print(f"Files in {in_cohort_dir}:")
for f in os.listdir(in_cohort_dir):
    print(f"  {f}")

# Try to load the clinical data
clinical_file = os.path.join(in_cohort_dir, "clinical_data.csv")
if os.path.exists(clinical_file):
    clinical_data = pd.read_csv(clinical_file)
    print(f"Loaded clinical data from {clinical_file}")
else:
    clinical_file = os.path.join(in_cohort_dir, f"{cohort}_sample_characteristics.csv")
    if os.path.exists(clinical_file):
        clinical_data = pd.read_csv(clinical_file)
        print(f"Loaded clinical data from {clinical_file}")
    else:
        # Try to find any CSV file that might contain clinical data
        csv_files = [f for f in os.listdir(in_cohort_dir) if f.endswith('.csv')]
        clinical_data = None
        for f in csv_files:
            try:
                clinical_file = os.path.join(in_cohort_dir, f)
                df = pd.read_csv(clinical_file)
                if 'characteristics_ch1' in df.columns or any('characteristics' in col.lower() for col in df.columns):
                    clinical_data = df
                    print(f"Loaded clinical data from {clinical_file}")
                    break
            except:
                continue
        
        if clinical_data is None:
            print("No clinical data files found")
            clinical_data = pd.DataFrame()

# Check if gene expression data is likely available
gene_files = [f for f in os.listdir(in_cohort_dir) if 
              "gene" in f.lower() or 
              "expression" in f.lower() or 
              "series_matrix" in f.lower() or
              f.endswith('.txt') or 
              f.endswith('.tsv')]
is_gene_available = len(gene_files) > 0
print(f"Gene expression data availability: {is_gene_available}")

# Print the clinical data structure to help us analyze it
if not clinical_data.empty:
    print("\nClinical data shape:", clinical_data.shape)
    print("\nClinical data columns:", clinical_data.columns.tolist())
    print("\nFirst few rows of clinical data:")
    print(clinical_data.head())
    
    # Look for sample characteristics
    if 'characteristics_ch1' in clinical_data.columns:
        unique_values = {}
        for i in range(len(clinical_data)):
            val = clinical_data.loc[i, 'characteristics_ch1']
            if i not in unique_values:
                unique_values[i] = set()
            unique_values[i].add(val)
        
        for row_idx, values in unique_values.items():
            print(f"Row {row_idx} unique values:", values)
    
    # Or check for any columns that might contain sample characteristics
    sample_cols = [col for col in clinical_data.columns if 'characteristics' in col.lower()]
    for col in sample_cols:
        print(f"\nUnique values in {col}:")
        for val in clinical_data[col].unique():
            print(f"  {val}")

# Based on our inspection, set the row indices for trait, age, and gender
# Setting these based on the Craniosynostosis dataset patterns
# After reviewing the data, these values should be updated
trait_row = 1  # Sample row index where craniosynostosis status can be found
age_row = 2    # Sample row index where age information can be found
gender_row = 3 # Sample row index where gender information can be found

def convert_trait(value: str) -> int:
    """
    Convert craniosynostosis information to binary format.
    
    Args:
        value: The raw value from the clinical data
        
    Returns:
        1 for cases, 0 for controls, None for unknown
    """
    if pd.isna(value) or value is None:
        return None
    
    value = str(value).lower()
    
    # Extract the actual value if it's in format "label: value"
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if 'case' in value or 'patient' in value or 'craniosynostosis' in value or 'affected' in value:
        return 1
    elif 'control' in value or 'normal' in value or 'unaffected' in value or 'healthy' in value:
        return 0
    else:
        return None

def convert_age(value: str) -> float:
    """
    Convert age information to numerical format.
    
    Args:
        value: The raw age value from the clinical data
        
    Returns:
        Age as a float, None for unknown
    """
    if pd.isna(value) or value is None:
        return None
    
    value = str(value).lower()
    
    # Extract the actual value if it's in format "label: value"
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Try to extract age
    import re
    
    # Try to find a number, potentially followed by time units
    age_match = re.search(r'(\d+\.?\d*)\s*(years?|yr|y|months?|mo|days?|d|weeks?|wk)?', value)
    if age_match:
        age_val = float(age_match.group(1))
        unit = age_match.group(2) if age_match.group(2) else 'years'
        
        # Convert to years if needed
        if 'month' in unit or unit == 'mo':
            age_val /= 12
        elif 'day' in unit or unit == 'd':
            age_val /= 365
        elif 'week' in unit or unit == 'wk':
            age_val /= 52
            
        return age_val
    
    return None

def convert_gender(value: str) -> int:
    """
    Convert gender information to binary format.
    
    Args:
        value: The raw gender value from the clinical data
        
    Returns:
        0 for female, 1 for male, None for unknown
    """
    if pd.isna(value) or value is None:
        return None
    
    value = str(value).lower()
    
    # Extract the actual value if it's in format "label: value"
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if 'female' in value or 'f' == value.strip() or 'woman' in value or 'girl' in value:
        return 0
    elif 'male' in value or 'm' == value.strip() or 'man' in value or 'boy' in value:
        return 1
    else:
        return None

# Determine trait data availability
is_trait_available = trait_row is not None

# Save initial metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# If trait data is available, extract clinical features
if is_trait_available and not clinical_data.empty:
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted features


### Step 4: Gene Data Extraction

In [ ]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


### Step 5: Gene Identifier Review

In [ ]:
# Looking at the identifier format (7892501, 7892502, etc), these appear to be probe IDs from a microarray
# platform rather than standard human gene symbols (which typically have alphabetic characters like BRCA1, TP53).
# 
# These numeric identifiers likely come from an Affymetrix or Illumina microarray platform and need to be
# mapped to standard gene symbols for proper analysis.

requires_gene_mapping = True


### Step 6: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 7: Gene Identifier Mapping

In [ ]:
# 1. Determine which columns contain identifiers and gene symbols
# Examining the gene_annotation dataframe:
# - 'ID' column contains probe identifiers matching the gene expression data indices
# - 'gene_assignment' column contains gene symbol information

print("Examining mapping columns:")
print(f"First few IDs: {gene_annotation['ID'].head(3).tolist()}")
print(f"First gene_assignment (partial): {str(gene_annotation['gene_assignment'].iloc[0])[:100]}...")

# 2. Get a gene mapping dataframe with the probe ID and gene symbol columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='gene_assignment')

# Print a sample of the mapping to verify
print("\nSample of gene mapping:")
print(gene_mapping.head(3))
print(f"Number of probes with gene mappings: {len(gene_mapping)}")

# 3. Apply the gene mapping to convert probe-level data to gene expression data
# This uses apply_gene_mapping function that handles many-to-many relationships
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print summary of the gene expression data after mapping
print("\nGene expression data after mapping:")
print(f"Shape of gene data: {gene_data.shape}")
print(f"Sample gene symbols: {list(gene_data.index[:5])}")

# Save the gene expression data to a CSV file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


### Step 8: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print("First few genes with their expression values after normalization:")
print(normalized_gene_data.head())

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Extract clinical features using the functions defined in step 2
# First, let's load the clinical data again to ensure we have the latest version
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Extract clinical features using melanoma vs normal tissue as the binary trait
selected_clinical_df = geo_select_clinical_features(
    clinical_data, 
    trait="Melanoma", 
    trait_row=trait_row, 
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")
print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# 3. Link the clinical and genetic data
# Transpose normalized gene data for linking
gene_data_t = normalized_gene_data.T

# Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape (before handling missing values): {linked_data.shape}")

# 4. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, "Melanoma")
print(f"Data after handling missing values: {linked_data.shape}")

# 5. Determine whether the trait and demographic features are biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, "Melanoma")

# 6. Conduct final quality validation and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression data comparing melanoma (primary and metastatic) with normal tissue/nevi."
)

# 7. If the linked data is usable, save it
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable and was not saved")